In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!pip install chainer

In [3]:
pip install numpy==1.25

In [4]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import numpy as np
import random


import argparse
import numpy as np

import chainer
import chainer.functions as F
import chainer.links as L

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training,serializers
from chainer.training import extensions
from PIL import Image

c:\Users\PC\anaconda3\envs\drac_baseline\lib\site-packages\chainer\_environment_check.py:72: UserWarning: 
--------------------------------------------------------------------------------
CuPy (cupy) version 13.2.0 may not be compatible with this version of Chainer.
Please consider installing the supported version by running:
  $ pip install 'cupy>=7.7.0,<8.0.0'

See the following page for more details:
  https://docs.cupy.dev/en/latest/install.html
--------------------------------------------------------------------------------

  warnings.warn(msg.format(


In [6]:
# Define location of the images and labels
xbd_dir="C:\\Users\\PC\\Desktop\\damage_assessement_data\\train"
images_dir=os.path.join(xbd_dir,"images")
labels_dir=os.path.join(xbd_dir,"labels")
masks_dir = os.path.join(xbd_dir, 'masks')
extracted_files = "C:\\Users\\PC\\Desktop\\drac_saved_files"
train_data_info=os.path.join(extracted_files,"train_data_info.csv")

train_batch_size = 2
test_batch_size = 2
epochs = 100
frequency = 25
log_dir = os.path.join(extracted_files,"logs")

In [7]:
# Read mask and convert to array
def read_mask_image(path, dtype):
    with Image.open(path) as img:
        #convert image to binary format
        img = img.convert('1')
        image = np.array(img, dtype=dtype)
    return image

In [8]:
# Read image and convert to array
def read_disaster_image(path, dtype):
    with Image.open(path) as img:
        image = np.array(img, dtype=dtype)
    return image

In [9]:
from chainer.dataset import dataset_mixin
class CustomDataset(dataset_mixin.DatasetMixin):
    def __init__(self, train_test_data_path, images_path, masks_path, image_dtype=np.float32, mask_dtype=np.int32, mean=0, crop_size=400, is_test=False):
        with open(train_test_data_path) as file:
            image_mask_pairs = []
            for line in file:
                line = line.rstrip('\n')
                image_filename = label_filename = line
                image_mask_pairs.append((image_filename, label_filename))
        self.image_mask_pairs = image_mask_pairs
        self.images_path = images_path
        self.masks_path = masks_path
        self.image_dtype = image_dtype
        self.mask_dtype = mask_dtype
        self.mean = mean[np.newaxis, np.newaxis, :]
        self.crop_size = crop_size
        self.is_test = is_test

    def __len__(self):
        return len(self.image_mask_pairs)

    def get_example(self, index):
        image_filename, mask_filename = self.image_mask_pairs[index]
        images_path = os.path.join(self.images_path, image_filename)
        image = read_disaster_image(images_path, self.image_dtype)
        image = (image - self.mean) / 255.0
        masks_path = os.path.join(self.masks_path, mask_filename)
        mask_image = read_mask_image(masks_path, self.mask_dtype)
        h, w, c = image.shape
        mask = np.zeros(shape=[h, w], dtype=np.int32)
        mask[mask_image > 0] = 1
        #compare the size of image to desired crop size
        #If image size is less than crop size add padding to ensure size requirement
        if (h < self.crop_size) or (w < self.crop_size):
            H, W = max(h, self.crop_size), max(w, self.crop_size)
            padding_x1, padding_y1 = (W - w) // 2, (H - h) // 2
            padding_x2, padding_y2 = (W - w - padding_x1), (H - h - padding_y1)
            image = np.pad(image, ((padding_y1, padding_y2), (padding_x1, padding_x2), (0, 0)), 'symmetric')
            if self.is_test:
                mask = np.pad(mask, ((padding_y1, padding_y2), (padding_x1, padding_x2)), 'constant', constant_values=255)
            else:
                mask = np.pad(mask, ((padding_y1, padding_y2), (padding_x1, padding_x2)), 'symmetric')
            h, w = H, W
            if not self.is_test:
                # Randomly flip horizontally
                if random.choice([True, False]):
                    image = image[:, ::-1, :]
                    mask = mask[:, ::-1]
                # Randomly flip vertically
                if random.choice([True, False]):
                    image = image[::-1, :, :]
                    mask = mask[::-1, :]
                # Randomly crop
                top = random.randint(0, h - self.crop_size)
                left = random.randint(0, w - self.crop_size)
            else:
                top = (h - self.crop_size) // 2
                left = (w - self.crop_size) // 2
            bottom = top + self.crop_size
            right = left + self.crop_size
            image = image[top:bottom, left:right]
            mask = mask[top:bottom, left:right]
            print(image.transpose(2, 0, 1), mask)
        return image.transpose(2, 0, 1), mask


In [10]:
import chainer
import chainer.functions as F
import chainer.links as L


class UNet(chainer.Chain):

    def __init__(self):
        super(UNet, self).__init__(
            c0=L.Convolution2D(3, 32, 3, 1, 1),
            c1=L.Convolution2D(32, 64, 4, 2, 1),
            c2=L.Convolution2D(64, 64, 3, 1, 1),
            c3=L.Convolution2D(64, 128, 4, 2, 1),
            c4=L.Convolution2D(128, 128, 3, 1, 1),
            c5=L.Convolution2D(128, 256, 4, 2, 1),
            c6=L.Convolution2D(256, 256, 3, 1, 1),
            c7=L.Convolution2D(256, 512, 4, 2, 1),
            c8=L.Convolution2D(512, 512, 3, 1, 1),

            dc8=L.Deconvolution2D(1024, 512, 4, 2, 1),
            dc7=L.Convolution2D(512, 256, 3, 1, 1),
            dc6=L.Deconvolution2D(512, 256, 4, 2, 1),
            dc5=L.Convolution2D(256, 128, 3, 1, 1),
            dc4=L.Deconvolution2D(256, 128, 4, 2, 1),
            dc3=L.Convolution2D(128, 64, 3, 1, 1),
            dc2=L.Deconvolution2D(128, 64, 4, 2, 1),
            dc1=L.Convolution2D(64, 32, 3, 1, 1),
            dc0=L.Convolution2D(64, 2, 3, 1, 1),
            bnc0=L.BatchNormalization(32),
            bnc1=L.BatchNormalization(64),
            bnc2=L.BatchNormalization(64),
            bnc3=L.BatchNormalization(128),
            bnc4=L.BatchNormalization(128),
            bnc5=L.BatchNormalization(256),
            bnc6=L.BatchNormalization(256),
            bnc7=L.BatchNormalization(512),
            bnc8=L.BatchNormalization(512),
            bnd8=L.BatchNormalization(512),
            bnd7=L.BatchNormalization(256),
            bnd6=L.BatchNormalization(256),
            bnd5=L.BatchNormalization(128),
            bnd4=L.BatchNormalization(128),
            bnd3=L.BatchNormalization(64),
            bnd2=L.BatchNormalization(64),
            bnd1=L.BatchNormalization(32)
        )


    def forward(self, x):
        x = x.astype(np.float32)
        e0 = F.relu(self.bnc0(self.c0(x)))
        e1 = F.relu(self.bnc1(self.c1(e0)))
        e2 = F.relu(self.bnc2(self.c2(e1)))
        del e1
        e3 = F.relu(self.bnc3(self.c3(e2)))
        e4 = F.relu(self.bnc4(self.c4(e3)))
        del e3
        e5 = F.relu(self.bnc5(self.c5(e4)))
        e6 = F.relu(self.bnc6(self.c6(e5)))
        del e5
        e7 = F.relu(self.bnc7(self.c7(e6)))
        e8 = F.relu(self.bnc8(self.c8(e7)))

        d8 = F.relu(self.bnd8(self.dc8(F.concat([e7, e8]))))
        del e7, e8
        d7 = F.relu(self.bnd7(self.dc7(d8)))
        del d8
        d6 = F.relu(self.bnd6(self.dc6(F.concat([e6, d7]))))
        del d7, e6
        d5 = F.relu(self.bnd5(self.dc5(d6)))
        del d6
        d4 = F.relu(self.bnd4(self.dc4(F.concat([e4, d5]))))
        del d5, e4
        d3 = F.relu(self.bnd3(self.dc3(d4)))
        del d4
        d2 = F.relu(self.bnd2(self.dc2(F.concat([e2, d3]))))
        del d3, e2
        d1 = F.relu(self.bnd1(self.dc1(d2)))
        del d2
        d0 = self.dc0(F.concat([e0, d1]))
        return d0


    def __call__(self, x, t):
        h = self.forward(x)
        loss = F.softmax_cross_entropy(h, t, ignore_label=255)
        accuracy = F.accuracy(h, t, ignore_label=255)
        chainer.report({'loss': loss, 'accuracy': accuracy}, self)
        return loss

In [24]:
# Initialize UNet()
model = UNet()

chainer.cuda.get_device_from_id(0).use()
model = model.to_gpu()
print(model)

UNet(
  (bnc0): BatchNormalization(size=32, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnc1): BatchNormalization(size=64, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnc2): BatchNormalization(size=64, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnc3): BatchNormalization(size=128, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnc4): BatchNormalization(size=128, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnc5): BatchNormalization(size=256, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnc6): BatchNormalization(size=256, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnc7): BatchNormalization(size=512, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnc8): BatchNormalization(size=512, decay=0.9, eps=2e-05, dtype=float32, use_gamma=True, use_beta=True),
  (bnd1): BatchNormalizat

In [14]:
# Setup Adam optimizer
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

In [15]:
# Load Train and Test Datasets
mean = np.load(os.path.join(extracted_files, "mean.npy"))

train_dataset = CustomDataset(os.path.join(extracted_files, "train.txt"), images_dir, masks_dir, mean=mean, crop_size=400, is_test=False)

test_dataset = CustomDataset (os.path.join(extracted_files, "test.txt"), images_dir, masks_dir, mean=mean, crop_size=480, is_test=True)

train_iterator = chainer.iterators.SerialIterator(train_dataset, train_batch_size,shuffle=True)
test_iterator = chainer.iterators.SerialIterator(test_dataset, test_batch_size, shuffle=False)

In [16]:
# Set up a trainer
updater = training.StandardUpdater(train_iterator, optimizer, device=0)
trainer = training.Trainer(updater, (epochs, 'epoch'), out=log_dir)

In [17]:
# Evaluate the model with the test dataset
trainer.extend(extensions.Evaluator(test_iterator, model, device=0))

c:\Users\PC\anaconda3\envs\drac_baseline\lib\site-packages\chainer\training\extensions\evaluator.py:130: UserWarning: The `repeat` property of the iterator {} 
  warnings.warn(msg)


In [18]:
# logging
trainer.extend(extensions.LogReport())

# Print report for each epoch
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'validation/main/loss','main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

# Print a progress bar
trainer.extend(extensions.ProgressBar())

In [20]:
import torch

print("Is CUDA available?", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("CUDA device name:", torch.cuda.get_device_name(0))

Is CUDA available? True
CUDA device count: 1
CUDA device name: NVIDIA GeForce RTX 3060


In [22]:
import torch

# Function to monitor GPU memory usage
def print_gpu_memory():
    allocated = torch.cuda.memory_allocated() / 1024**3
    reserved = torch.cuda.memory_reserved() / 1024**3
    print(f"Allocated: {allocated:.2f} GB, Reserved: {reserved:.2f} GB")

print_gpu_memory()


Allocated: 0.00 GB, Reserved: 0.00 GB


In [19]:
# Run the trainer
trainer.run()

: 

In [ ]:
model_save_path = 'C:\\Users\\PC\\Desktop\\drac_saved_files\\localization_model'
chainer.serializers.save_npz(model_save_path, model)